In [122]:

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier

codes are from here:
https://medium.com/@hjhuney/implementing-a-random-forest-classification-model-in-python-583891c99652

In [123]:

dataset = pd.read_csv('/Users/zhangbincai/Desktop/study/code club/machine-learning-pipelines-r/data/data.tsv', sep='\t')

cancerStatus = np.array(dataset.loc[:, dataset.columns == 'cancer'])

otus= np.array(dataset.loc[: , dataset.columns != 'cancer'])

# print (cancerStatus)

In [124]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_cancerStatus, test_cancerStatus,train_otus, test_otus  = train_test_split(cancerStatus, otus, test_size = 0.25, random_state = 42)

In [125]:
print('Training Features Shape:', train_otus.shape)
print('Training Labels Shape:', train_cancerStatus.shape)
print('Testing Features Shape:', test_otus.shape)
print('Testing Labels Shape:', test_cancerStatus.shape)

Training Features Shape: (219, 5206)
Training Labels Shape: (219, 1)
Testing Features Shape: (73, 5206)
Testing Labels Shape: (73, 1)


In [126]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()

rfc.fit(train_otus, train_cancerStatus.ravel())

rfc_predict = rfc.predict(test_otus)



In [127]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

rfc_cv_score = cross_val_score(rfc, otus, cancerStatus.ravel(), cv=10, scoring='roc_auc')

In [128]:
print("=== Confusion Matrix ===")
print(confusion_matrix(test_cancerStatus, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_cancerStatus, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[45  3]
 [14 11]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.76      0.94      0.84        48
           1       0.79      0.44      0.56        25

    accuracy                           0.77        73
   macro avg       0.77      0.69      0.70        73
weighted avg       0.77      0.77      0.75        73



=== All AUC Scores ===
[0.79398148 0.79861111 0.59313725 0.75980392 0.85294118 0.57843137
 0.7622549  0.94362745 0.62745098 0.83333333]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7543572984749454


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(train_otus, train_cancerStatus.ravel())
# print results
print(rfc_random.best_params_)

In [130]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=220, max_features='auto')
rfc.fit(train_otus,train_cancerStatus.ravel())
rfc_predict = rfc.predict(test_otus)
rfc_cv_score = cross_val_score(rfc, otus, cancerStatus.ravel(), cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(test_cancerStatus.ravel(), rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(test_cancerStatus.ravel(), rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[44  4]
 [13 12]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.77      0.92      0.84        48
           1       0.75      0.48      0.59        25

    accuracy                           0.77        73
   macro avg       0.76      0.70      0.71        73
weighted avg       0.76      0.77      0.75        73



=== All AUC Scores ===
[0.87962963 0.875      0.64705882 0.77941176 0.76960784 0.6004902
 0.74754902 0.91176471 0.64705882 0.78431373]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.7641884531590414


In [131]:
test= pd.read_csv('/Users/zhangbincai/Desktop/study/code club/machine-learning-pipelines-r/data/predata.tsv', sep='\t')

testotus= np.array(test.loc[: , test.columns != 'cancer'])

print(rfc.predict(testotus))




[0 0 1 1 1 0 1]
